## Statsmodel Forecast with Wallaroo Features: Deploy and Test Infer

This tutorial series demonstrates how to use Wallaroo to deploy a statsmodel ARIMA forecast model and perform sample inferences through it.

In the previous step "Statsmodel Forecast with Wallaroo Features: Model Creation", the statsmodel was trained and saved to the Python file `forecast.py`.  This file will now be uploaded to a Wallaroo instance as a Python model, then used for sample inferences.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [42]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import pyarrow as pa

In [43]:
wallaroo.__version__

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [44]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

# wallarooPrefix = "product-uat-ee."
# wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [45]:
# used for unique connection names

# import string
# import random

# suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='john'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.

In [46]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload Model

The Python model created in "Forecast and Parallel Infer with Statsmodel: Model Creation" will now be uploaded.  Note that the Framework is set to `Framework.PYTHON` to inform the Wallaroo engine on what model framework is being added.

The other versions of the model are also uploaded in this step to show how to hot swap between models and other uses.

In [47]:
# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
control_model_file = './models/forecast_standard_df.py'

challenger01_model_name = 'forecast-challenger01-model'
challenger01_model_file = './models/forecast_alternate01.py'

challenger02_model_name = 'forecast-challenger02-model'
challenger02_model_file = './models/forecast_alternate02.py'

# Holding on these for later
input_schema = pa.schema([
    pa.field('count', pa.list_(pa.int64()))
])

output_schema = pa.schema([
    pa.field('forecast', pa.list_(pa.int64())),
    pa.field('weekly_average', pa.list_(pa.float64()))
])

# upload the models

bike_day_model = (wl.upload_model(control_model_name, 
                                 control_model_file, 
                                 framework=Framework.PYTHON)
                                 .configure("python", 
                                 input_schema=input_schema, 
                                 output_schema=output_schema)
                )

challenger_model_01 = (wl.upload_model(challenger01_model_name, 
                                 challenger01_model_file, 
                                 framework=Framework.PYTHON)
                                 .configure("python", 
                                 input_schema=input_schema, 
                                 output_schema=output_schema)
                )

challenger_model_02 = (wl.upload_model(challenger02_model_name, 
                                 challenger02_model_file, 
                                 framework=Framework.PYTHON)
                                 .configure("python", 
                                 input_schema=input_schema, 
                                 output_schema=output_schema)
                )



### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

Until a pipeline is deployed, the steps assigned to it only exist in the local memory.  During deployment the pipeline steps, configurations, and other details are set in the database, and resources allocated from the cluster for the pipeline's use.

In [48]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)

pipeline.deploy(deployment_config = deploy_config)
# pipeline.deploy()

name,forecast-workshop-pipeline
created,2023-08-02 15:50:59.480547+00:00
last_updated,2023-08-02 15:50:59.480547+00:00
deployed,True
tags,
versions,1d89287b-4eff-47ec-a7bb-8cedaac1f33f
steps,forecast-control-model


In [49]:
pipeline.steps()

[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': 'ffca51bd-f9c6-40cf-a36b-c6126ce98dd3', 'sha': 'dcbd11947ae1e51f5c882687a0ec2dbcf60c0b0de8e5156cb6f1d669e0a6d76b'}]}}]

### Run Inference

Run a test inference to verify the pipeline is operational from the sample test data stored in `./data/testdata_dict.json`.

In [50]:
import datetime
inference_start = datetime.datetime.now()

inferencedata = pd.read_json("./data/testdata_standard.df.json")
display(inferencedata)

results = pipeline.infer(inferencedata)

display(results)

,count
0,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 1605, 1538, 1746, 1472, 1589, 1913, 1815, 2115, 2475, 2927, 1635, 1812, 1107, 1450, 1917, 1807, 1461, 1969, 2402, 1446, 1851]"


,time,in.count,out.forecast,out.weekly_average,check_failures
0,2023-08-02 15:51:22.916,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 1605, 1538, 1746, 1472, 1589, 1913, 1815, 2115, 2475, 2927, 1635, 1812, 1107, 1450, 1917, 1807, 1461, 1969, 2402, 1446, 1851]","[1764, 1749, 1743, 1741, 1740, 1740, 1740]",1745.285714,0


### Hot Swap Model

Models are "hot swapped" - aka the pipeline step for the model is replaced with another model - without undeploying the pipeline.  During the hot swap, a model in a pipeline step is "swapped" with another model.  Incoming inferences are cached during the milliseconds it takes to update the step with the new model, then submitted to the pipeline with the new step.

To replace a pipeline step, use the Pipeline `replace_with_model_step(index, model)`, where `index` is the step number ordered from zero, and the `model` is the model to be replacing it with.  The pipeline is then deployed - even if currently already deployed - to store the updated settings and update the pipeline with resource allocation, etc.

Once the model is swapped out, we will perform another sample inference to test the difference in output.

In [51]:
pipeline.replace_with_model_step(0, challenger_model_01)

pipeline.deploy()

display(pipeline.steps())

results = pipeline.infer(inferencedata)

display(results)
inference_end = datetime.datetime.now()

[{'ModelInference': {'models': [{'name': 'forecast-challenger01-model', 'version': '5bb81da9-f6ce-4d09-8435-9b27b0846c00', 'sha': '77d1045ee551cb101435be265344b4483d53892e67cc0a20a70cf7e2ccfdd4a0'}]}}]

,time,in.count,out.forecast,out.weekly_average,check_failures
0,2023-08-02 15:51:47.963,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 1605, 1538, 1746, 1472, 1589, 1913, 1815, 2115, 2475, 2927, 1635, 1812, 1107, 1450, 1917, 1807, 1461, 1969, 2402, 1446, 1851]","[1703, 1757, 1737, 1744, 1742, 1743, 1742]",1738.285714,0


### Pipeline Logs

Logs are displayed with the Pipeline `logs()` method.  This displays the log for the current version.  Adding the parameter `dataset=["time", "out.json","metadata"]` lets us get the metadata parameter `metadata.last_model` to show what model was used for the inference request.

#### Pipeline Logs References

[Wallaroo SDK Essentials Guide: Pipeline Log Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-logs/)

In [52]:
display(pipeline.logs(start_datetime=inference_start, end_datetime=inference_end, dataset=["time", "out.forecast", "out.weekly_average","metadata"]) \
        .loc[:, ["time", "out.forecast", "out.weekly_average", "metadata.last_model"]])


,time,out.forecast,out.weekly_average,metadata.last_model
0,2023-08-02 15:51:47.963,"[1703, 1757, 1737, 1744, 1742, 1743, 1742]",1738.285714,"{""model_name"":""forecast-challenger01-model"",""model_sha"":""77d1045ee551cb101435be265344b4483d53892e67cc0a20a70cf7e2ccfdd4a0""}"
1,2023-08-02 15:51:22.916,"[1764, 1749, 1743, 1741, 1740, 1740, 1740]",1745.285714,"{""model_name"":""forecast-control-model"",""model_sha"":""dcbd11947ae1e51f5c882687a0ec2dbcf60c0b0de8e5156cb6f1d669e0a6d76b""}"


### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [53]:
pipeline.undeploy()

name,forecast-workshop-pipeline
created,2023-08-02 15:50:59.480547+00:00
last_updated,2023-08-02 15:51:23.810302+00:00
deployed,False
tags,
versions,"5ec5e8dc-7492-498b-9652-b3733e4c87f7, 1d89287b-4eff-47ec-a7bb-8cedaac1f33f"
steps,forecast-control-model
